## Jupyter notebook to create graphs using data from different linux monitoring tools

In [1]:
import matplotlib.pyplot as plt
import fileinput
import numpy as np
import re

In [2]:
def read_file(file_name):
    lines = list()
    for line in fileinput.input(file_name):
        lines.append(line)
    return lines

In [3]:
def get_tests(lines):
    test = list()
    tests = list()
    
    for i in lines:
        if len(re.findall("#test",i)) > 0:
            tests.append(test)
            test = list()
        else:
            test.append(i)
            
    tests.append(test)
    
    return tests[:4]

In [4]:
def get_memory_values(lines):
    kbmemused = list()
    memusedpercent = list()
    for i in lines:
        _,_,_,kbmemused_,memusedpercent_,_,_,_,_ = i.rstrip().split()
        kbmemused.append(float(kbmemused_))
        memusedpercent.append(float(memusedpercent_))
        
    return (kbmemused,memusedpercent)

In [5]:
def get_memory(lines):    
    kbs = list()
    percents = list()
    for i in get_tests(lines):
        kbmemused,memusedpercent = get_memory_values(i)
        kbmemused,memusedpercent = (max(kbmemused),max(memusedpercent))
        kbs.append(kbmemused)
        percents.append(memusedpercent)
        
    kbs = sorted(kbs)
    percents = sorted(percents)
    
    
    return (sum(kbs[:3])/3,sum(percents[:3])/3)

In [6]:
def get_cpu_times(lines,repetitions):
    idle = list()
    iowait = list()
    user = list()
    system = list()
    maximum = list()
    for i in lines:
        ##print(i.rstrip().split())
        _,_,_,user_,_,system_,iowait_,_,idle_= i.rstrip().split()
        iowait.append(float(iowait_))
        idle.append(float(idle_))
        user.append(float(user_))
        system.append(float(system_))
        maximum.append(100 - float(idle_))
    return (user,system,idle,iowait,maximum)

In [7]:
def get_cpu(lines,repetitions):
    totals = list()
    idles = list()
    iowaits = list()
    users = list()
    systems = list()
    maximums = list()
    for i in get_tests(lines):
        user,system,idle,iowait,maximum = get_cpu_times(i,repetitions)
        total,user,system,idle,iowait,maximum = (float(len(i))/repetitions,sum(user)*0.01/repetitions,
                                         sum(system)*0.01/repetitions,sum(idle)*0.01/repetitions,sum(iowait)*0.01/repetitions,
                                        max(maximum))
        totals.append(total)
        idles.append(idle)
        iowaits.append(iowait)
        users.append(user)
        systems.append(system)
        maximums.append(maximum)
    
    ##print(totals)
    totals = sorted(totals)
    idles = sorted(idles)
    iowaits = sorted(iowaits)
    users = sorted(users)
    systems = sorted(systems)
    maximums= sorted(maximums)
    
    return (sum(totals[:3])/3,sum(users[:3])/3,sum(systems[:3])/3,sum(idles[:3])/3,sum(iowaits[:3])/3,sum(maximums[:3])/3)
    

In [8]:
def get_io_values(lines):
    tps = list()
    breads = list()
    bwrtns = list()
    for i in lines:
        _,_,tps_,_,_,breads_,bwrtns_ = i.rstrip().split()
        tps.append(float(tps_))
        breads.append(float(breads_))
        bwrtns.append(float(bwrtns_))
        
    return (tps,breads,bwrtns)

In [9]:
def get_io(lines):
    network = list()
    disk = list()
    for i in get_tests(lines):
        tps,breads,bwrtns = get_io_values(i)
        tps,bwreads  = (sum(tps),sum(breads) + sum(bwrtns) )
        network.append(tps)
        disk.append(bwreads)
    
    network = sorted(network)
    disk = sorted(disk)
    
    return (sum(network[:3])/3,sum(disk[:3])/3)

In [10]:
time,user,system,idle,iowait,maximum = (0,1,2,3,4,5)
graphs = ["time","user","system","idle","iowait","maximum"]

def cpu_times(file,repetitions):
    lines = read_file(file)[3:]
    return get_cpu(lines,repetitions)

def create_graphs_cpu(labels,files,repetitions,saveloc,graph=time,ybottom=0):
    values = list()
    print(len(labels))
    print(len(repetitions))
    for x in range(len(labels)):
        
        print(files[x])
        values.append(cpu_times(files[x],repetitions[x]))  
    
    x = np.arange(len(labels))
    
    values = [x[graph] for x in values]

    print(values)

    fig, ax = plt.subplots(figsize=(17,5))
   
    plt.bar(x,values)
    if(ybottom != 0):
        ax.set_ylim(bottom=ybottom)
    plt.xticks(x,tuple(labels))
    ##plt.show()
    plt.savefig(saveloc)
    plt.close()
    

In [11]:
def create_graphs_cpu_part1(directory,test,size,saveloc,repetitions=1,graph=time,ybottom=0):
    compilers = [('gnuO3','5.3.0'),('gnuO2','5.3.0'),('gnuO1','5.3.0'),
    ('gnuO3','6.1.0'),('gnuO2','6.1.0'),('gnuO1','6.1.0'),
    ('gnuO3','7.2.0'),('gnuO2','7.2.0'),('gnuO1','7.2.0'),
    ('intelO3','2019'),('intelO2','2019'),('intelO1','2019')]
    
    repetitions = [repetitions for x in range(len(compilers))]
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_cpu(labels,files,repetitions,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["B","C"]
sizes_omp = ["C","D"]
repetitions_ser = [1,1]
repetitions_omp = [10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
        for size in range(len(sizes_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_cpu_part1(f"esc-tests/part1/{machines[machine]}/omp/cpu/",
                                        tests_omp[test],sizes_omp[size],f"images/part1/{machines[machine]}/omp/cpu/",
                                        repetitions_omp[size],graph)
                
                create_graphs_cpu_part1(f"esc-tests/part1/{machines[machine]}/ser/cpu/",
                                        tests_ser[test],sizes_ser[size],f"images/part1/{machines[machine]}/ser/cpu/",
                                        repetitions_ser[size],graph)

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r662/omp/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO2.5.3.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO1.5.3.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO3.6.1.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO2.6.1.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO1.6.1.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO3.7.2.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO2.7.2.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO1.7.2.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.intelO3.2019.txt', 'esc-tests/part1/r662/omp/cpu/is.C.intelO2.2019.txt', 'esc-tests/part1/r662/omp/cpu/is.C.intelO1.2019.txt']
12
12
esc-tests/part1/r662/omp/cpu/is.C.gnuO3.5.3.0.txt
esc-tests/part1/r662/omp/cpu/is.C.gnuO2.5.3.0.txt
esc-tests/part1/r662/omp/cpu/is.C.gnuO1.5.3.0.txt
esc-tests/part1/r662

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r662/omp/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO2.5.3.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO1.5.3.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO3.6.1.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO2.6.1.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO1.6.1.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO3.7.2.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO2.7.2.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.gnuO1.7.2.0.txt', 'esc-tests/part1/r662/omp/cpu/is.C.intelO3.2019.txt', 'esc-tests/part1/r662/omp/cpu/is.C.intelO2.2019.txt', 'esc-tests/part1/r662/omp/cpu/is.C.intelO1.2019.txt']
12
12
esc-tests/part1/r662/omp/cpu/is.C.gnuO3.5.3.0.txt
esc-tests/part1/r662/omp/cpu/is.C.gnuO2.5.3.0.txt
esc-tests/part1/r662/omp/cpu/is.C.gnuO1.5.3.0.txt
esc-tests/part1/r662

esc-tests/part1/r662/ser/cpu/is.B.gnuO1.5.3.0.txt
esc-tests/part1/r662/ser/cpu/is.B.gnuO3.6.1.0.txt
esc-tests/part1/r662/ser/cpu/is.B.gnuO2.6.1.0.txt
esc-tests/part1/r662/ser/cpu/is.B.gnuO1.6.1.0.txt
esc-tests/part1/r662/ser/cpu/is.B.gnuO3.7.2.0.txt
esc-tests/part1/r662/ser/cpu/is.B.gnuO2.7.2.0.txt
esc-tests/part1/r662/ser/cpu/is.B.gnuO1.7.2.0.txt
esc-tests/part1/r662/ser/cpu/is.B.intelO3.2019.txt
esc-tests/part1/r662/ser/cpu/is.B.intelO2.2019.txt
esc-tests/part1/r662/ser/cpu/is.B.intelO1.2019.txt
[2.4666666666666686, 2.5333333333333363, 2.6433333333333358, 2.4899999999999998, 2.5899999999999985, 2.5933333333333337, 2.4366666666666674, 2.4333333333333322, 2.3699999999999997, 2.6366666666666654, 2.616666666666665, 2.469999999999999]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r662/omp/cpu/is.D.gnuO3.5.3.0.txt', 'esc-tests/part1/r66

esc-tests/part1/r662/ser/cpu/is.C.gnuO2.5.3.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO1.5.3.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO3.6.1.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO2.6.1.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO1.6.1.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO3.7.2.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO2.7.2.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO1.7.2.0.txt
esc-tests/part1/r662/ser/cpu/is.C.intelO3.2019.txt
esc-tests/part1/r662/ser/cpu/is.C.intelO2.2019.txt
esc-tests/part1/r662/ser/cpu/is.C.intelO1.2019.txt
[0.03746666666666668, 0.04833333333333334, 0.053533333333333315, 0.04933333333333332, 0.04836666666666666, 0.049166666666666664, 0.03153333333333335, 0.039633333333333354, 0.03093333333333335, 0.04886666666666666, 0.03970000000000002, 0.03646666666666668]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-t

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r662/ser/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO2.5.3.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO1.5.3.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO3.6.1.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO2.6.1.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO1.6.1.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO3.7.2.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO2.7.2.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.gnuO1.7.2.0.txt', 'esc-tests/part1/r662/ser/cpu/is.C.intelO3.2019.txt', 'esc-tests/part1/r662/ser/cpu/is.C.intelO2.2019.txt', 'esc-tests/part1/r662/ser/cpu/is.C.intelO1.2019.txt']
12
12
esc-tests/part1/r662/ser/cpu/is.C.gnuO3.5.3.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO2.5.3.0.txt
esc-tests/part1/r662/ser/cpu/is.C.gnuO1.5.3.0.txt
esc-tests/part1/r662

...

esc-tests/part1/r641/ser/cpu/mg.C.gnuO1.6.1.0.txt
esc-tests/part1/r641/ser/cpu/mg.C.gnuO3.7.2.0.txt
esc-tests/part1/r641/ser/cpu/mg.C.gnuO2.7.2.0.txt
esc-tests/part1/r641/ser/cpu/mg.C.gnuO1.7.2.0.txt
esc-tests/part1/r641/ser/cpu/mg.C.intelO3.2019.txt
esc-tests/part1/r641/ser/cpu/mg.C.intelO2.2019.txt
esc-tests/part1/r641/ser/cpu/mg.C.intelO1.2019.txt
[0.013633333333333332, 0.014166666666666668, 0.0134, 0.010266666666666667, 9.999999999999999e-05, 9.999999999999999e-05, 0.012033333333333333, 0.0126, 0.014199999999999999, 0.0311, 0.00019999999999999998, 0.027166666666666662]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r641/omp/cpu/ep.D.gnuO3.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/ep.D.gnuO2.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/ep.D.gnuO1.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/ep.D.gnuO3.6.1.0.txt', 'esc-tests/part1/r641/o

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.7.2.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.7.2.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.7.2.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.intelO3.2019.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.intelO2.2019.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.intelO1.2019.txt']
12
12
esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt
esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.5.3.0.txt
esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.5.3.0.txt
esc-tests/part1/r641

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.7.2.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.7.2.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.7.2.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.intelO3.2019.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.intelO2.2019.txt', 'esc-tests/part1/r641/omp/cpu/mg.C.intelO1.2019.txt']
12
12
esc-tests/part1/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt
esc-tests/part1/r641/omp/cpu/mg.C.gnuO2.5.3.0.txt
esc-tests/part1/r641/omp/cpu/mg.C.gnuO1.5.3.0.txt
esc-tests/part1/r641

esc-tests/part1/r641/ser/cpu/cg.C.gnuO2.7.2.0.txt
esc-tests/part1/r641/ser/cpu/cg.C.gnuO1.7.2.0.txt
esc-tests/part1/r641/ser/cpu/cg.C.intelO3.2019.txt
esc-tests/part1/r641/ser/cpu/cg.C.intelO2.2019.txt
esc-tests/part1/r641/ser/cpu/cg.C.intelO1.2019.txt
[12.946333333333394, 7.624099999999994, 7.558299999999992, 7.537533333333325, 7.656499999999991, 9.688700000000027, 10.84770000000001, 7.580299999999991, 7.773299999999991, 9.907433333333321, 12.696333333333413, 9.502699999999983]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r641/omp/cpu/mg.D.gnuO3.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.D.gnuO2.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.D.gnuO1.5.3.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.D.gnuO3.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.D.gnuO2.6.1.0.txt', 'esc-tests/part1/r641/omp/cpu/mg.D.gnuO1.6.1.0.txt', 'esc-tests/pa

esc-tests/part1/r641/omp/cpu/mg.D.gnuO1.7.2.0.txt
esc-tests/part1/r641/omp/cpu/mg.D.intelO3.2019.txt
esc-tests/part1/r641/omp/cpu/mg.D.intelO2.2019.txt
esc-tests/part1/r641/omp/cpu/mg.D.intelO1.2019.txt
[0.0041333333333333335, 0.005833333333333334, 0.0031000000000000003, 0.0005, 0.0016333333333333332, 0.0016333333333333332, 0.0, 0.0023666666666666667, 0.0008999999999999999, 0.007766666666666667, 0.0037333333333333333, 0.0044666666666666665]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/part1/r641/ser/cpu/cg.C.gnuO3.5.3.0.txt', 'esc-tests/part1/r641/ser/cpu/cg.C.gnuO2.5.3.0.txt', 'esc-tests/part1/r641/ser/cpu/cg.C.gnuO1.5.3.0.txt', 'esc-tests/part1/r641/ser/cpu/cg.C.gnuO3.6.1.0.txt', 'esc-tests/part1/r641/ser/cpu/cg.C.gnuO2.6.1.0.txt', 'esc-tests/part1/r641/ser/cpu/cg.C.gnuO1.6.1.0.txt', 'esc-tests/part1/r641/ser/cpu/cg.C.gnuO3.7.2.0.txt',

In [12]:
def create_graphs_cpu_part1(mpi_hybrid,directory,test,size,saveloc,repetitions=1,graph=time,ybottom=0):
    
    if(mpi_hybrid == "mpi"):
        compilers = [('gnu_eth','1.8.2'),
                    ('gnu_mpich2_eth','1.5'),('gnu_eth','1.6.3'),
                    ('intel_mpich2_eth','1.5'),('intel_mpich_eth','1.2.7'),
                    ('intel_eth','1.8.2'),('intel_eth','1.6.3')]
    
    if(mpi_hybrid == "hybrid"):
        compilers = [('gnu_eth','1.8.4'),('gnu_eth','1.8.2'),
                    ('intel_eth','1.8.2'),('intel_eth','1.6.3')]
    
    repetitions = [repetitions for x in range(len(compilers))]
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_cpu(labels,files,repetitions,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_mpi = ["is","ep","mg"]
tests_hybrid = ['sp-mz','bt-mz']
sizes_mpi = ["C","D"]
sizes_hybrid = ["A","B"]
repetitions_mpi = [1,1]
repetitions_hybrid = [1,1]

for machine in range(len(machines)):
    for test in range(len(tests_mpi)):
        for size in range(len(sizes_mpi)):
            for graph in range(len(graphs)):
                
                create_graphs_cpu_part1("mpi",f"esc-tests/part1/{machines[machine]}/mpi/cpu/",
                                        tests_mpi[test],sizes_mpi[size],f"images/part1/{machines[machine]}/mpi/cpu/",
                                        repetitions_mpi[size],graph)
                
                if(test < len(tests_hybrid)):
                    create_graphs_cpu_part1("hybrid",f"esc-tests/part1/{machines[machine]}/hybrid/cpu/",
                                            tests_hybrid[test],sizes_hybrid[size],f"images/part1/{machines[machine]}/hybrid/cpu/",
                                            repetitions_hybrid[size],graph)

['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.6.3.txt']
7
7
esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt
esc-tests/part1/r662/mpi/cpu/is.C.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.6.3.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.8.2.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.6.3.txt
[3.3333333333333335, 2.6666666666666665, 3.66666666666

['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.6.3.txt']
7
7
esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt
esc-tests/part1/r662/mpi/cpu/is.C.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/cpu/is.C.gnu_eth.1.6.3.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.8.2.txt
esc-tests/part1/r662/mpi/cpu/is.C.intel_eth.1.6.3.txt
[66.77666666666667, 66.74, 66.79333333333334, 66.98333

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/hybrid/cpu/sp-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part1/r662/hybrid/cpu/sp-mz.B.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/cpu/sp-mz.B.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/cpu/sp-mz.B.intel_eth.1.6.3.txt']
4
4
esc-tests/part1/r662/hybrid/cpu/sp-mz.B.gnu_eth.1.8.4.txt
esc-tests/part1/r662/hybrid/cpu/sp-mz.B.gnu_eth.1.8.2.txt
esc-tests/part1/r662/hybrid/cpu/sp-mz.B.intel_eth.1.8.2.txt
esc-tests/part1/r662/hybrid/cpu/sp-mz.B.intel_eth.1.6.3.txt
[0.05499999999999999, 0.04723333333333333, 0.031266666666666665, 0.029866666666666663]
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/cpu/is.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/is.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/is.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/cpu/is.D.i

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/hybrid/cpu/bt-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part1/r662/hybrid/cpu/bt-mz.A.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/cpu/bt-mz.A.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/cpu/bt-mz.A.intel_eth.1.6.3.txt']
4
4
esc-tests/part1/r662/hybrid/cpu/bt-mz.A.gnu_eth.1.8.4.txt
esc-tests/part1/r662/hybrid/cpu/bt-mz.A.gnu_eth.1.8.2.txt
esc-tests/part1/r662/hybrid/cpu/bt-mz.A.intel_eth.1.8.2.txt
esc-tests/part1/r662/hybrid/cpu/bt-mz.A.intel_eth.1.6.3.txt
[31.683999999999997, 31.16556666666666, 9.691366666666667, 9.691533333333334]
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/cpu/ep.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/ep.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/ep.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/cpu/ep.C.intel_mpic

['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/cpu/ep.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/ep.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/ep.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/cpu/ep.D.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/cpu/ep.D.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r662/mpi/cpu/ep.D.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/cpu/ep.D.intel_eth.1.6.3.txt']
7
7
esc-tests/part1/r662/mpi/cpu/ep.D.gnu_eth.1.8.2.txt
esc-tests/part1/r662/mpi/cpu/ep.D.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/cpu/ep.D.gnu_eth.1.6.3.txt
esc-tests/part1/r662/mpi/cpu/ep.D.intel_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/cpu/ep.D.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r662/mpi/cpu/ep.D.intel_eth.1.8.2.txt
esc-tests/part1/r662/mpi/cpu/ep.D.intel_eth.1.6.3.txt
[70.85416666666664, 60.01316666666668, 61.565466666666

...

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/hybrid/cpu/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part1/r641/hybrid/cpu/sp-mz.A.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/cpu/sp-mz.A.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/cpu/sp-mz.A.intel_eth.1.6.3.txt']
4
4
esc-tests/part1/r641/hybrid/cpu/sp-mz.A.gnu_eth.1.8.4.txt
esc-tests/part1/r641/hybrid/cpu/sp-mz.A.gnu_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/cpu/sp-mz.A.intel_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/cpu/sp-mz.A.intel_eth.1.6.3.txt
[5.410000000000001, 4.88, 24.866666666666664, 24.996666666666666]
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/cpu/is.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/is.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/is.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/cpu/is.D.intel_mpich2_eth.1.5.t

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/hybrid/cpu/sp-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part1/r641/hybrid/cpu/sp-mz.B.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/cpu/sp-mz.B.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/cpu/sp-mz.B.intel_eth.1.6.3.txt']
4
4
esc-tests/part1/r641/hybrid/cpu/sp-mz.B.gnu_eth.1.8.4.txt
esc-tests/part1/r641/hybrid/cpu/sp-mz.B.gnu_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/cpu/sp-mz.B.intel_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/cpu/sp-mz.B.intel_eth.1.6.3.txt
[0.05456666666666666, 0.00039999999999999996, 0.0003, 0.043100000000000006]
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/cpu/is.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/is.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/is.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/cpu/is.D.intel_mpich2

[26.029200000000003, 26.783766666666665, 7.335633333333333, 7.391199999999999]
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/cpu/ep.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/ep.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/ep.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/cpu/ep.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/ep.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r641/mpi/cpu/ep.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/ep.C.intel_eth.1.6.3.txt']
7
7
esc-tests/part1/r641/mpi/cpu/ep.C.gnu_eth.1.8.2.txt
esc-tests/part1/r641/mpi/cpu/ep.C.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r641/mpi/cpu/ep.C.gnu_eth.1.6.3.txt
esc-tests/part1/r641/mpi/cpu/ep.C.intel_mpich2_eth.1.5.txt
esc-tests/part1/r641/mpi/cpu/ep.C.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r641/mpi/cpu/ep.C.intel_eth.1.8.2.txt
esc-tests/part1/r641/mpi/cpu/

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/hybrid/cpu/bt-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part1/r641/hybrid/cpu/bt-mz.B.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/cpu/bt-mz.B.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/cpu/bt-mz.B.intel_eth.1.6.3.txt']
4
4
esc-tests/part1/r641/hybrid/cpu/bt-mz.B.gnu_eth.1.8.4.txt
esc-tests/part1/r641/hybrid/cpu/bt-mz.B.gnu_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/cpu/bt-mz.B.intel_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/cpu/bt-mz.B.intel_eth.1.6.3.txt
[109.71953333333336, 86.85286666666666, 28.357366666666667, 28.256133333333334]
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/cpu/ep.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/ep.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/ep.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/cpu/ep.D.intel_mp

['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/cpu/mg.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/mg.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/mg.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/cpu/mg.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/cpu/mg.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r641/mpi/cpu/mg.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/cpu/mg.C.intel_eth.1.6.3.txt']
7
7
esc-tests/part1/r641/mpi/cpu/mg.C.gnu_eth.1.8.2.txt
esc-tests/part1/r641/mpi/cpu/mg.C.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r641/mpi/cpu/mg.C.gnu_eth.1.6.3.txt
esc-tests/part1/r641/mpi/cpu/mg.C.intel_mpich2_eth.1.5.txt
esc-tests/part1/r641/mpi/cpu/mg.C.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r641/mpi/cpu/mg.C.intel_eth.1.8.2.txt
esc-tests/part1/r641/mpi/cpu/mg.C.intel_eth.1.6.3.txt
[0.0028666666666666667, 0.005699999999999999, 0.007266

In [13]:
def create_graphs_cpu_part2(directory,test,sizes,saveloc,repetitions=1,graph=time,ybottom=0):
    compiler = 'gnuO3.5.3.0'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(repetitions)
    print(files)
    create_graphs_cpu(labels,files,repetitions,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["S","W","A","B","C"]
sizes_omp = ["S","W","A","B","C","D"]
repetitions_ser = [100,10,10,1,1]
repetitions_omp = [100,100,100,100,10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_cpu_part2(f"esc-tests/part2/{machines[machine]}/omp/cpu/",
                                        tests_omp[test],sizes_omp,f"images/part2/{machines[machine]}/omp/cpu/",
                                        repetitions_omp,graph)
                create_graphs_cpu_part2(f"esc-tests/part2/{machines[machine]}/ser/cpu/",
                                            tests_ser[test],sizes_ser,f"images/part2/{machines[machine]}/ser/cpu/",
                                            repetitions_ser,graph)

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r662/omp/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.D.gnuO3.5.3.0.txt']
6
6
esc-tests/part2/r662/omp/cpu/is.S.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/is.W.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/is.A.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/is.B.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/is.C.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/is.D.gnuO3.5.3.0.txt
[0.12, 0.14333333333333334, 0.1366666666666667, 0.4166666666666667, 1.5333333333333332, 51.0]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r662/ser/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r

esc-tests/part2/r662/omp/cpu/ep.W.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/ep.A.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/ep.B.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/ep.C.gnuO3.5.3.0.txt
esc-tests/part2/r662/omp/cpu/ep.D.gnuO3.5.3.0.txt
[0.11666666666666665, 0.16, 0.6866666666666666, 2.353333333333333, 8.666666666666666, 128.66666666666666]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r662/ser/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.C.gnuO3.5.3.0.txt']
5
5
esc-tests/part2/r662/ser/cpu/mg.S.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/mg.W.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/mg.A.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/mg.B.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/mg.C.gnuO3.5.3.0.txt
[0.01, 0.3, 2.5666666666666664, 6.333333333333333, 56.333333333333336]
['S', '

[100, 10, 10, 1, 1]
['esc-tests/part2/r662/ser/cpu/cg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/cg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/cg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/cg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/cg.C.gnuO3.5.3.0.txt']
5
5
esc-tests/part2/r662/ser/cpu/cg.S.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/cg.W.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/cg.A.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/cg.B.gnuO3.5.3.0.txt
esc-tests/part2/r662/ser/cpu/cg.C.gnuO3.5.3.0.txt
[0.06, 0.3, 1.3, 104.33333333333333, 469.6666666666667]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r662/omp/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/mg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/mg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/mg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/mg.D.gnuO3.5.3.0.txt']
6
6
esc-tests/part2/r662/o

esc-tests/part2/r641/omp/cpu/is.W.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/is.A.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/is.B.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/is.C.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/is.D.gnuO3.5.3.0.txt
[0.10666666666666667, 0.06, 0.22333333333333336, 0.51, 2.0, 60.666666666666664]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r641/ser/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.C.gnuO3.5.3.0.txt']
5
5
esc-tests/part2/r641/ser/cpu/is.S.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/is.W.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/is.A.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/is.B.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/is.C.gnuO3.5.3.0.txt
[0.01, 0.13333333333333333, 2.066666666666667, 7.333333333333333, 32.0]
['S', 'W', 'A', 'B', 'C', 'D']
[

esc-tests/part2/r641/omp/cpu/ep.C.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/ep.D.gnuO3.5.3.0.txt
[0.11333333333333333, 0.16333333333333333, 0.89, 3.1966666666666668, 12.0, 179.0]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r641/ser/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.C.gnuO3.5.3.0.txt']
5
5
esc-tests/part2/r641/ser/cpu/mg.S.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/mg.W.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/mg.A.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/mg.B.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/mg.C.gnuO3.5.3.0.txt
[0.01, 0.3666666666666667, 2.8666666666666667, 7.333333333333333, 56.333333333333336]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r641/omp/cpu/ep.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/ep.W.gnuO3.5.3.0.txt', 

esc-tests/part2/r641/ser/cpu/cg.W.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/cg.A.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/cg.B.gnuO3.5.3.0.txt
esc-tests/part2/r641/ser/cpu/cg.C.gnuO3.5.3.0.txt
[0.056666666666666664, 0.3333333333333333, 1.5666666666666667, 132.0, 239.0]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r641/omp/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/mg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/mg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/mg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/mg.D.gnuO3.5.3.0.txt']
6
6
esc-tests/part2/r641/omp/cpu/mg.S.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/mg.W.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/mg.A.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/mg.B.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt
esc-tests/part2/r641/omp/cpu/mg.D.gnuO3.5.3.0.txt
[0.30275433333333335, 0.20198533333333332

In [12]:
def create_graphs_cpu_part2(mpi_hybrid,directory,test,sizes,saveloc,repetitions=1,graph=time,ybottom=0):
    
    if(mpi_hybrid == "mpi"):
        compiler = 'gnu_eth.1.8.2'   
        
    if(mpi_hybrid == "hybrid"):
        compiler = 'gnu_eth.1.8.4'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(repetitions)
    print(files)
    create_graphs_cpu(labels,files,repetitions,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_mpi = ["is","ep","mg"]
tests_hybrid = ['sp-mz','bt-mz']
sizes_mpi = ["S","W","A","B","C","D"]
sizes_hybrid = ["S","W","A","B","C"]
repetitions_mpi = [100,100,10,10,1,1]
repetitions_hybrid = [10,1,1,1,1]

for machine in range(len(machines)):
    for test in range(len(tests_mpi)):
            for graph in range(len(graphs)):
                
                create_graphs_cpu_part2("mpi",f"esc-tests/part2/{machines[machine]}/mpi/cpu/",
                                        tests_mpi[test],sizes_mpi,f"images/part2/{machines[machine]}/mpi/cpu/",
                                        repetitions_mpi,graph)
                
                if(test < len(tests_hybrid)):
                    create_graphs_cpu_part2("hybrid",f"esc-tests/part2/{machines[machine]}/hybrid/cpu/",
                                            tests_hybrid[test],sizes_hybrid,f"images/part2/{machines[machine]}/hybrid/cpu/",
                                            repetitions_hybrid,graph)

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 10, 10, 1, 1]
['esc-tests/part2/r662/mpi/cpu/is.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/is.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/is.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/is.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/is.D.gnu_eth.1.8.2.txt']
6
6
esc-tests/part2/r662/mpi/cpu/is.S.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.W.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.A.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.B.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.D.gnu_eth.1.8.2.txt
[0.7033333333333333, 0.7433333333333333, 0.7666666666666666, 1.3, 3.0, 55.666666666666664]
['S', 'W', 'A', 'B', 'C']
[10, 1, 1, 1, 1]
['esc-tests/part2/r662/hybrid/cpu/sp-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/sp-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/sp-mz.A

esc-tests/part2/r662/mpi/cpu/is.A.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.B.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.C.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/is.D.gnu_eth.1.8.2.txt
[40.263333333333335, 59.99, 41.85333333333333, 59.35666666666666, 66.80333333333333, 67.26]
['S', 'W', 'A', 'B', 'C']
[10, 1, 1, 1, 1]
['esc-tests/part2/r662/hybrid/cpu/sp-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/sp-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/sp-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/sp-mz.C.gnu_eth.1.8.4.txt']
5
5
esc-tests/part2/r662/hybrid/cpu/sp-mz.S.gnu_eth.1.8.4.txt
esc-tests/part2/r662/hybrid/cpu/sp-mz.W.gnu_eth.1.8.4.txt
esc-tests/part2/r662/hybrid/cpu/sp-mz.A.gnu_eth.1.8.4.txt
esc-tests/part2/r662/hybrid/cpu/sp-mz.B.gnu_eth.1.8.4.txt
esc-tests/part2/r662/hybrid/cpu/sp-mz.C.gnu_eth.1.8.4.txt
[2.770000000000001, 3.276666666666666, 3.7900000000000014

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 10, 10, 1, 1]
['esc-tests/part2/r662/mpi/cpu/ep.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/ep.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/ep.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/ep.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/ep.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/cpu/ep.D.gnu_eth.1.8.2.txt']
6
6
esc-tests/part2/r662/mpi/cpu/ep.S.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/ep.W.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/ep.A.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/ep.B.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/ep.C.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/cpu/ep.D.gnu_eth.1.8.2.txt
[40.00666666666667, 40.81333333333333, 60.43666666666667, 67.20666666666666, 67.06666666666666, 67.19333333333333]
['S', 'W', 'A', 'B', 'C']
[10, 1, 1, 1, 1]
['esc-tests/part2/r662/hybrid/cpu/bt-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/cpu/bt-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 10, 10, 1, 1]
['esc-tests/part2/r641/mpi/cpu/is.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/is.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/is.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/is.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/is.D.gnu_eth.1.8.2.txt']
6
6
esc-tests/part2/r641/mpi/cpu/is.S.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/is.W.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/is.A.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/is.B.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/is.C.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/is.D.gnu_eth.1.8.2.txt
[0.10937866666666668, 0.11449866666666665, 0.11761333333333333, 0.13109, 0.13133333333333333, 0.4354666666666667]
['S', 'W', 'A', 'B', 'C']
[10, 1, 1, 1, 1]
['esc-tests/part2/r641/hybrid/cpu/sp-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/cpu/sp-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/

['S', 'W', 'A', 'B', 'C']
[10, 1, 1, 1, 1]
['esc-tests/part2/r641/hybrid/cpu/bt-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/cpu/bt-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/cpu/bt-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/cpu/bt-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/cpu/bt-mz.C.gnu_eth.1.8.4.txt']
5
5
esc-tests/part2/r641/hybrid/cpu/bt-mz.S.gnu_eth.1.8.4.txt
esc-tests/part2/r641/hybrid/cpu/bt-mz.W.gnu_eth.1.8.4.txt
esc-tests/part2/r641/hybrid/cpu/bt-mz.A.gnu_eth.1.8.4.txt
esc-tests/part2/r641/hybrid/cpu/bt-mz.B.gnu_eth.1.8.4.txt
esc-tests/part2/r641/hybrid/cpu/bt-mz.C.gnu_eth.1.8.4.txt
[0.013646666666666668, 0.18766666666666668, 1.3677, 4.918266666666668, 20.506966666666674]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 10, 10, 1, 1]
['esc-tests/part2/r641/mpi/cpu/ep.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/ep.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/ep.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/ep.B.gnu_eth.1

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 10, 10, 1, 1]
['esc-tests/part2/r641/mpi/cpu/mg.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/mg.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/mg.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/mg.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/mg.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/mg.D.gnu_eth.1.8.2.txt']
6
6
esc-tests/part2/r641/mpi/cpu/mg.S.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/mg.W.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/mg.A.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/mg.B.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/mg.C.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/cpu/mg.D.gnu_eth.1.8.2.txt
[0.11329566666666667, 0.11298166666666672, 0.12651, 0.12909333333333342, 0.21946666666666667, 1.177533333333329]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 10, 10, 1, 1]
['esc-tests/part2/r641/mpi/cpu/mg.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/cpu/mg.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/

In [13]:
kb,per = (0,1)
graphs = ["kb","per"]

def memory_values(file):
    lines = read_file(file)[3:]
    return get_memory(lines)

def create_graphs_memory(labels,files,saveloc,graph=kb,ybottom=0):
    values = list()
    for x in range(len(labels)):
        print(files[x])
        values.append(memory_values(files[x]))  
    
    x = np.arange(len(labels))
    
    values = [x[graph] for x in values]
    

    fig, ax = plt.subplots(figsize=(17,5))
    plt.bar(x,values)
    if(ybottom != 0):
        ax.set_ylim(bottom=ybottom)
    plt.xticks(x,tuple(labels))
    plt.savefig(saveloc)
    plt.close()

In [16]:
def create_graphs_memory_part1(directory,test,size,saveloc,graph=time,ybottom=0):
    compilers = [('gnuO3','5.3.0'),('gnuO2','5.3.0'),('gnuO1','5.3.0'),
    ('gnuO3','6.1.0'),('gnuO2','6.1.0'),('gnuO1','6.1.0'),
    ('gnuO3','7.2.0'),('gnuO2','7.2.0'),('gnuO1','7.2.0'),
    ('intelO3','2019'),('intelO2','2019'),('intelO1','2019')]
    
    
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_memory(labels,files,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ["is","mg","cg"]
sizes_ser = ["B","C"]
sizes_omp = ["C","D"]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
        for size in range(len(sizes_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_memory_part1(f"esc-tests/{machines[machine]}/omp/memory/",
                                        tests_omp[test],sizes_omp[size],
                                        f"images/{machines[machine]}/omp/memory/",graph)
               
                create_graphs_memory_part1(f"esc-tests/{machines[machine]}/ser/memory/",
                                        tests_ser[test],sizes_ser[size],
                                        f"images/{machines[machine]}/ser/memory/",graph)

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/memory/is.C.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/memory/is.C.intelO3.2019.txt', 'esc-tests/r662/omp/memory/is.C.intelO2.2019.txt', 'esc-tests/r662/omp/memory/is.C.intelO1.2019.txt']
esc-tests/r662/omp/memory/is.C.gnuO3.5.3.0.txt


FileNotFoundError: [Errno 2] No such file or directory: 'esc-tests/r662/omp/memory/is.C.gnuO3.5.3.0.txt'

In [14]:
def create_graphs_memory_part1(mpi_hybrid,directory,test,size,saveloc,graph=time,ybottom=0):
    
    
    if(mpi_hybrid == "mpi"):
        compilers = [('gnu_eth','1.8.2'),
                    ('gnu_mpich2_eth','1.5'),('gnu_eth','1.6.3'),
                    ('intel_mpich2_eth','1.5'),('intel_mpich_eth','1.2.7'),
                    ('intel_eth','1.8.2'),('intel_eth','1.6.3')]
    
    if(mpi_hybrid == "hybrid"):
        compilers = [('gnu_eth','1.8.4'),('gnu_eth','1.8.2'),
                    ('intel_eth','1.8.2'),('intel_eth','1.6.3')]
    
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_memory(labels,files,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)
    
machines = ["r662","r641"]
tests_mpi = ["is","ep","mg"]
tests_hybrid = ['sp-mz','bt-mz']
sizes_mpi = ["C","D"]
sizes_hybrid = ["A","B"]
repetitions_mpi = [1,1]
repetitions_hybrid = [1,1]

for machine in range(len(machines)):
    for test in range(len(tests_mpi)):
        for size in range(len(sizes_mpi)):
            for graph in range(len(graphs)):
                
                create_graphs_memory_part1("mpi",f"esc-tests/part1/{machines[machine]}/mpi/memory/",
                                        tests_mpi[test],sizes_mpi[size],f"images/part1/{machines[machine]}/mpi/memory/",
                                        graph)
                
                if(test < len(tests_hybrid)):
                    create_graphs_memory_part1("hybrid",f"esc-tests/part1/{machines[machine]}/hybrid/memory/",
                                            tests_hybrid[test],sizes_hybrid[size],f"images/part1/{machines[machine]}/hybrid/memory/",
                                            graph)



['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/memory/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/memory/is.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/memory/is.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/memory/is.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/memory/is.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r662/mpi/memory/is.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/memory/is.C.intel_eth.1.6.3.txt']
esc-tests/part1/r662/mpi/memory/is.C.gnu_eth.1.8.2.txt
esc-tests/part1/r662/mpi/memory/is.C.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/memory/is.C.gnu_eth.1.6.3.txt
esc-tests/part1/r662/mpi/memory/is.C.intel_mpich2_eth.1.5.txt
esc-tests/part1/r662/mpi/memory/is.C.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r662/mpi/memory/is.C.intel_eth.1.8.2.txt
esc-tests/part1/r662/mpi/memory/is.C.intel_eth.1.6.3.txt
['gnu_eth.1.8.4'

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/hybrid/memory/bt-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part1/r662/hybrid/memory/bt-mz.A.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/memory/bt-mz.A.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/memory/bt-mz.A.intel_eth.1.6.3.txt']
esc-tests/part1/r662/hybrid/memory/bt-mz.A.gnu_eth.1.8.4.txt
esc-tests/part1/r662/hybrid/memory/bt-mz.A.gnu_eth.1.8.2.txt
esc-tests/part1/r662/hybrid/memory/bt-mz.A.intel_eth.1.8.2.txt
esc-tests/part1/r662/hybrid/memory/bt-mz.A.intel_eth.1.6.3.txt
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/memory/ep.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/memory/ep.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/memory/ep.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/memory/ep.D.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/mem

['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/hybrid/memory/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part1/r641/hybrid/memory/sp-mz.A.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/memory/sp-mz.A.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/memory/sp-mz.A.intel_eth.1.6.3.txt']
esc-tests/part1/r641/hybrid/memory/sp-mz.A.gnu_eth.1.8.4.txt
esc-tests/part1/r641/hybrid/memory/sp-mz.A.gnu_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/memory/sp-mz.A.intel_eth.1.8.2.txt
esc-tests/part1/r641/hybrid/memory/sp-mz.A.intel_eth.1.6.3.txt
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/memory/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/memory/is.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/memory/is.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/memory/is.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/mem

['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/memory/ep.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/memory/ep.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/memory/ep.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/memory/ep.D.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/memory/ep.D.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r641/mpi/memory/ep.D.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/memory/ep.D.intel_eth.1.6.3.txt']
esc-tests/part1/r641/mpi/memory/ep.D.gnu_eth.1.8.2.txt
esc-tests/part1/r641/mpi/memory/ep.D.gnu_mpich2_eth.1.5.txt
esc-tests/part1/r641/mpi/memory/ep.D.gnu_eth.1.6.3.txt
esc-tests/part1/r641/mpi/memory/ep.D.intel_mpich2_eth.1.5.txt
esc-tests/part1/r641/mpi/memory/ep.D.intel_mpich_eth.1.2.7.txt
esc-tests/part1/r641/mpi/memory/ep.D.intel_eth.1.8.2.txt
esc-tests/part1/r641/mpi/memory/ep.D.intel_eth.1.6.3.txt
['gnu_eth.1.8.4'

In [ ]:
def create_graphs_memory_part2(directory,test,sizes,saveloc,graph=time,ybottom=0):
    compiler = 'gnuO3.5.3.0'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(files)
    create_graphs_memory(labels,files,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["S","W","A","B","C"]
sizes_omp = ["S","W","A","B","C","D"]
repetitions_ser = [100,10,10,1,1]
repetitions_omp = [100,100,100,100,10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_memory_part2(f"esc-tests/part2/{machines[machine]}/omp/memory/",
                                        tests_omp[test],sizes_omp,f"images/part2/{machines[machine]}/omp/memory/",
                                        graph)
                create_graphs_memory_part2(f"esc-tests/part2/{machines[machine]}/ser/memory/",
                                            tests_ser[test],sizes_ser,f"images/part2/{machines[machine]}/ser/memory/",
                                            graph)

In [15]:
def create_graphs_memory_part2(mpi_hybrid,directory,test,sizes,saveloc,graph=time,ybottom=0):
    if(mpi_hybrid == "mpi"):
        compiler = 'gnu_eth.1.8.2'   
        
    if(mpi_hybrid == "hybrid"):
        compiler = 'gnu_eth.1.8.4'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(files)
    print(graph)
    print(graphs)
    create_graphs_memory(labels,files,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_mpi = ["is","ep","mg"]
tests_hybrid = ['sp-mz','bt-mz']
sizes_mpi = ["S","W","A","B","C","D"]
sizes_hybrid = ["S","W","A","B","C"]
repetitions_mpi = [100,100,10,10,1,1]
repetitions_hybrid = [10,1,1,1,1]

for machine in range(len(machines)):
    for test in range(len(tests_mpi)):
            for graph in range(len(graphs)):
                
                create_graphs_memory_part2("mpi",f"esc-tests/part2/{machines[machine]}/mpi/memory/",
                                        tests_mpi[test],sizes_mpi,f"images/part2/{machines[machine]}/mpi/memory/",
                                        graph)
                
                if(test < len(tests_hybrid)):
                    create_graphs_memory_part2("hybrid",f"esc-tests/part2/{machines[machine]}/hybrid/memory/",
                                            tests_hybrid[test],sizes_hybrid,f"images/part2/{machines[machine]}/hybrid/memory/",
                                            graph)

['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/mpi/memory/is.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/memory/is.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/memory/is.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/memory/is.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/memory/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/memory/is.D.gnu_eth.1.8.2.txt']
0
['kb', 'per']
esc-tests/part2/r662/mpi/memory/is.S.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/memory/is.W.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/memory/is.A.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/memory/is.B.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/memory/is.C.gnu_eth.1.8.2.txt
esc-tests/part2/r662/mpi/memory/is.D.gnu_eth.1.8.2.txt
['S', 'W', 'A', 'B', 'C']
['esc-tests/part2/r662/hybrid/memory/sp-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/memory/sp-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/memory/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/memory/sp-mz.B.gnu_eth.1.8

['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r641/mpi/memory/is.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/memory/is.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/memory/is.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/memory/is.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/memory/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r641/mpi/memory/is.D.gnu_eth.1.8.2.txt']
1
['kb', 'per']
esc-tests/part2/r641/mpi/memory/is.S.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/memory/is.W.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/memory/is.A.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/memory/is.B.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/memory/is.C.gnu_eth.1.8.2.txt
esc-tests/part2/r641/mpi/memory/is.D.gnu_eth.1.8.2.txt
['S', 'W', 'A', 'B', 'C']
['esc-tests/part2/r641/hybrid/memory/sp-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/memory/sp-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/memory/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part2/r641/hybrid/memory/sp-mz.B.gnu_eth.1.8

In [16]:
tcp,disk = (0,1)
graphs = ["tcp","disk"]

def io_values(file):
    lines = read_file(file)[3:]
    return get_io(lines)

def create_graphs_io(labels,files,saveloc,graph=kb,ybottom=0):
    values = list()
    for x in range(len(labels)):
        values.append(io_values(files[x]))  
    
    x = np.arange(len(labels))
    
    values = [x[graph] for x in values]
    

    fig, ax = plt.subplots(figsize=(17,5))
    
    plt.bar(x,values)
    if(ybottom != 0):
        ax.set_ylim(bottom=ybottom)
    plt.xticks(x,tuple(labels))
    plt.savefig(saveloc)
    plt.close()

In [ ]:
def create_graphs_io_part1(directory,test,size,saveloc,graph=time,ybottom=0):
    compilers = [('gnuO3','5.3.0'),('gnuO2','5.3.0'),('gnuO1','5.3.0'),
    ('gnuO3','6.1.0'),('gnuO2','6.1.0'),('gnuO1','6.1.0'),
    ('gnuO3','7.2.0'),('gnuO2','7.2.0'),('gnuO1','7.2.0'),
    ('intelO3','2019'),('intelO2','2019'),('intelO1','2019')]
    print(directory)

    
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_io(labels,files,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ["is","cg","mg"]
sizes_ser = ["B","C"]
sizes_omp = ["C","D"]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
        for size in range(len(sizes_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_io_part1(f"esc-tests/{machines[machine]}/omp/io/",
                                    tests_omp[test],sizes_omp[size],
                                    f"images/{machines[machine]}/omp/io/",graph)
                
                create_graphs_io_part1(f"esc-tests/{machines[machine]}/ser/io/",
                                    tests_ser[test],sizes_ser[size],
                                    f"images/{machines[machine]}/ser/io/",graph)

In [17]:
def create_graphs_io_part1(mpi_hybrid,directory,test,size,saveloc,graph=time,ybottom=0):
    
    
     
    if(mpi_hybrid == "mpi"):
        compilers = [('gnu_eth','1.8.2'),
                    ('gnu_mpich2_eth','1.5'),('gnu_eth','1.6.3'),
                    ('intel_mpich2_eth','1.5'),('intel_mpich_eth','1.2.7'),
                    ('intel_eth','1.8.2'),('intel_eth','1.6.3')]
    
    if(mpi_hybrid == "hybrid"):
        compilers = [('gnu_eth','1.8.4'),('gnu_eth','1.8.2'),
                    ('intel_eth','1.8.2'),('intel_eth','1.6.3')]
    
    print(directory)

    
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_io(labels,files,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_mpi = ["is","ep","mg"]
tests_hybrid = ['sp-mz','bt-mz']
sizes_mpi = ["C","D"]
sizes_hybrid = ["A","B"]
repetitions_mpi = [1,1]
repetitions_hybrid = [1,1]

for machine in range(len(machines)):
    for test in range(len(tests_mpi)):
        for size in range(len(sizes_mpi)):
            for graph in range(len(graphs)):
                
                create_graphs_io_part1("mpi",f"esc-tests/part1/{machines[machine]}/mpi/io/",
                                        tests_mpi[test],sizes_mpi[size],f"images/part1/{machines[machine]}/mpi/io/",
                                        graph)
                
                if(test < len(tests_hybrid)):
                    create_graphs_io_part1("hybrid",f"esc-tests/part1/{machines[machine]}/hybrid/io/",
                                            tests_hybrid[test],sizes_hybrid[size],f"images/part1/{machines[machine]}/hybrid/io/",
                                            graph)

esc-tests/part1/r662/mpi/io/
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/io/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/io/is.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/io/is.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/io/is.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/io/is.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r662/mpi/io/is.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/io/is.C.intel_eth.1.6.3.txt']
esc-tests/part1/r662/hybrid/io/
['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/hybrid/io/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part1/r662/hybrid/io/sp-mz.A.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/io/sp-mz.A.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/hybrid/io/sp-mz.A.intel_eth.1.6.3.txt']
esc-tests/part1/r662/mpi/io/
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_

esc-tests/part1/r662/mpi/io/
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/io/mg.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/io/mg.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/io/mg.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/io/mg.D.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/io/mg.D.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r662/mpi/io/mg.D.intel_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/io/mg.D.intel_eth.1.6.3.txt']
esc-tests/part1/r662/mpi/io/
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r662/mpi/io/mg.D.gnu_eth.1.8.2.txt', 'esc-tests/part1/r662/mpi/io/mg.D.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/mpi/io/mg.D.gnu_eth.1.6.3.txt', 'esc-tests/part1/r662/mpi/io/mg.D.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r662/

esc-tests/part1/r641/hybrid/io/
['gnu_eth.1.8.4', 'gnu_eth.1.8.2', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/hybrid/io/bt-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part1/r641/hybrid/io/bt-mz.B.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/io/bt-mz.B.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/hybrid/io/bt-mz.B.intel_eth.1.6.3.txt']
esc-tests/part1/r641/mpi/io/
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_eth.1.6.3', 'intel_mpich2_eth.1.5', 'intel_mpich_eth.1.2.7', 'intel_eth.1.8.2', 'intel_eth.1.6.3']
['esc-tests/part1/r641/mpi/io/mg.C.gnu_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/io/mg.C.gnu_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/io/mg.C.gnu_eth.1.6.3.txt', 'esc-tests/part1/r641/mpi/io/mg.C.intel_mpich2_eth.1.5.txt', 'esc-tests/part1/r641/mpi/io/mg.C.intel_mpich_eth.1.2.7.txt', 'esc-tests/part1/r641/mpi/io/mg.C.intel_eth.1.8.2.txt', 'esc-tests/part1/r641/mpi/io/mg.C.intel_eth.1.6.3.txt']
esc-tests/part1/r641/mpi/io/
['gnu_eth.1.8.2', 'gnu_mpich2_eth.1.5', 'gnu_

In [ ]:
def create_graphs_io_part2(directory,test,sizes,saveloc,graph=time,ybottom=0):
    compiler = 'gnuO3.5.3.0'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(files)
    create_graphs_io(labels,files,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["S","W","A","B","C"]
sizes_omp = ["S","W","A","B","C","D"]
repetitions_ser = [100,10,10,1,1]
repetitions_omp = [100,100,100,100,10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_io_part2(f"esc-tests/part2/{machines[machine]}/omp/io/",
                                        tests_omp[test],sizes_omp,f"images/part2/{machines[machine]}/omp/io/",
                                        graph)
                create_graphs_io_part2(f"esc-tests/part2/{machines[machine]}/ser/io/",
                                            tests_ser[test],sizes_ser,f"images/part2/{machines[machine]}/ser/io/",
                                            graph)

In [18]:
def create_graphs_io_part2(mpi_hybrid,directory,test,sizes,saveloc,graph=time,ybottom=0):

    if(mpi_hybrid == "mpi"):
        compiler = 'gnu_eth.1.8.2'   
        
    if(mpi_hybrid == "hybrid"):
        compiler = 'gnu_eth.1.8.4'    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(files)
    create_graphs_io(labels,files,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_mpi = ["is","ep","mg"]
tests_hybrid = ['sp-mz','bt-mz']
sizes_mpi = ["S","W","A","B","C","D"]
sizes_hybrid = ["S","W","A","B","C"]
repetitions_mpi = [100,100,10,10,1,1]
repetitions_hybrid = [10,1,1,1,1]

for machine in range(len(machines)):
    for test in range(len(tests_mpi)):
            for graph in range(len(graphs)):
                
                create_graphs_io_part2("mpi",f"esc-tests/part2/{machines[machine]}/mpi/io/",
                                        tests_mpi[test],sizes_mpi,f"images/part2/{machines[machine]}/mpi/io/",
                                        graph)
                
                if(test < len(tests_hybrid)):
                    create_graphs_io_part2("hybrid",f"esc-tests/part2/{machines[machine]}/hybrid/io/",
                                            tests_hybrid[test],sizes_hybrid,f"images/part2/{machines[machine]}/hybrid/io/",
                                            graph)

['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/mpi/io/is.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.D.gnu_eth.1.8.2.txt']
['S', 'W', 'A', 'B', 'C']
['esc-tests/part2/r662/hybrid/io/sp-mz.S.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/io/sp-mz.W.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/io/sp-mz.A.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/io/sp-mz.B.gnu_eth.1.8.4.txt', 'esc-tests/part2/r662/hybrid/io/sp-mz.C.gnu_eth.1.8.4.txt']
['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/mpi/io/is.S.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.W.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.A.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.B.gnu_eth.1.8.2.txt', 'esc-tests/part2/r662/mpi/io/is.C.gnu_eth.1.8.2.txt', 'esc-tests/part

In [ ]:
lines_cpu_gnuO1 = read_file('tests/omp/cpu/mg_D_gnuO1.txt')
lines_cpu_gnuO2 = read_file('tests/omp/cpu/mg_D_gnuO2.txt')
lines_cpu_gnuO3 = read_file('tests/omp/cpu/mg_D_gnuO3.txt')

lines_memory_gnuO1 = read_file('tests/omp/memory/mg_D_gnuO1.txt')
lines_memory_gnuO2 = read_file('tests/omp/memory/mg_D_gnuO2.txt')
lines_memory_gnuO3 = read_file('tests/omp/memory/mg_D_gnuO3.txt')

lines_io_gnuO1 = read_file('tests/omp/io/mg_D_gnuO1.txt')
lines_io_gnuO2 = read_file('tests/omp/io/mg_D_gnuO2.txt')
lines_io_gnuO3 = read_file('tests/omp/io/mg_D_gnuO3.txt')

In [ ]:
print(get_cpu(lines_cpu_gnuO1[3:],1))
print(get_cpu(lines_cpu_gnuO2[3:],1))
print(get_cpu(lines_cpu_gnuO3[3:],1))

print(get_memory(lines_memory_gnuO1[3:]))
print(get_memory(lines_memory_gnuO2[3:]))
print(get_memory(lines_memory_gnuO3[3:]))

print(get_io(lines_io_gnuO1[3:]))
print(get_io(lines_io_gnuO2[3:]))
print(get_io(lines_io_gnuO3[3:]))

In [ ]:
linesO3 = read_file('tests/omp/cpu/ep_D_gnuO3.txt')
linesO2 = read_file('tests/omp/cpu/ep_D_gnuO2.txt')
linesO1 = read_file('tests/omp/cpu/ep_D_gnuO1.txt')


In [ ]:
O3 = [len(x) for x in get_tests(linesO3[3:])]
O2 = [len(x) for x in get_tests(linesO2[3:])]
O1 = [len(x) for x in get_tests(linesO1[3:])]

print(sum(O1))
print(sum(O2))
print(sum(O3))